In [7]:
import pandas as pd
import numpy as np

data = pd.read_csv("./data/loan_sanction_train.csv")

print(type(data))

data.head()

# Cleaning the data #
data["Gender"] = data["Gender"].fillna("Male")
data["Married"] = data["Married"].fillna("No")
data["Dependents"] = data["Dependents"].fillna("0")
data["Self_Employed"] = data["Self_Employed"].fillna("No")
data["LoanAmount"] = data["LoanAmount"].fillna(data["LoanAmount"].mean())
data["Loan_Amount_Term"] = data["Loan_Amount_Term"].fillna(data["Loan_Amount_Term"].mean())
data["Credit_History"] = data["Credit_History"].fillna("1")


# Remove the Loan_ID Column
data = data.drop(columns="Loan_ID")

# making gender a normalization
male = data['Gender'].value_counts(normalize=True)['Male']

data_gender_normalized = data['Gender'].map({'Male': male, 'Female': 1 - male, "NaN": male})

data['Gender'] = data_gender_normalized

# normalizing education
education = data['Education'].value_counts(normalize=True)["Graduate"]

data_education_normalized = data['Education'].map({'Graduate': education, 'Not Graduate': 1 - education})

data['Education'] = data_education_normalized

# normalizing self_employed
self_employed = data['Self_Employed'].value_counts(normalize=True)["Yes"]

data_selfEmp_normalized = data['Self_Employed'].map({"Yes": self_employed, "No": 1-self_employed})

data["Self_Employed"] = data_selfEmp_normalized

# normalzing property area
area_property = data["Property_Area"].value_counts(normalize=True)

print(area_property)

data_area_normalized = data["Property_Area"].map({"Urban": area_property["Urban"], "Semiurban": area_property['Semiurban'], "Rural": area_property["Rural"]})

data["Property_Area"] = data_area_normalized

# normalzing married 
married = data["Married"].value_counts(normalize=True)["Yes"]

print(married)

married_normalized = data["Married"].map({"Yes": married, "No": 1-married})

data["Married"] = married_normalized

# Switchin loan status to 0 or 1
data["Loan_Status"] = data["Loan_Status"].map({"Y": 1, "N": 0})

# Cleaning the dependents
data["Dependents"] = data["Dependents"].map({"0":0, "1":1, "2":2, "3+": 3})

print(data.isna().sum())

data_train = data.loc[:, data.columns != "Loan_Status"]
data_target = data["Loan_Status"]


<class 'pandas.core.frame.DataFrame'>
Semiurban    0.379479
Urban        0.328990
Rural        0.291531
Name: Property_Area, dtype: float64
0.6482084690553745
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64


In [8]:
def ratio(a, b):
  return a/b

data['WealthToLoanRatio'] = data.apply(lambda row : ratio(row['ApplicantIncome'], row['LoanAmount']), axis = 1)

In [12]:
X = data_train.to_numpy()
y = data_target.to_numpy()

X

array([[0.8175895765472313, 0.35179153094462545, 0, ..., 360.0, 1.0,
        0.3289902280130293],
       [0.8175895765472313, 0.6482084690553745, 1, ..., 360.0, 1.0,
        0.2915309446254072],
       [0.8175895765472313, 0.6482084690553745, 0, ..., 360.0, 1.0,
        0.3289902280130293],
       ...,
       [0.8175895765472313, 0.6482084690553745, 1, ..., 360.0, 1.0,
        0.3289902280130293],
       [0.8175895765472313, 0.6482084690553745, 2, ..., 360.0, 1.0,
        0.3289902280130293],
       [0.1824104234527687, 0.35179153094462545, 0, ..., 360.0, 0.0,
        0.3794788273615635]], dtype=object)

In [15]:
from sklearn.linear_model import LinearRegression

model = LinearRegression().fit(X, y)

r_sq = model.score(X, y)

print(r_sq)

0.3226836027339236